# **Galactic R-velations: Classifying Celestial Objects with K-Nearest Neighbors**

# **Table of Contents**

### Introduction

> What is Stellar Classification?
> 
> Classifying Celestial Objects & Our Question
> 
> Dataset
> 
> Literature Review

### Methods & Results

> 1. Load Data
> 
> 2. Clean & Wrangle Data
> 
> 3. Exploratory Data Analysis
> 
> 4. Visualize Relationships
> 
> 5. Data Analysis

### Results

### Discussion

> Summary
> 
> Expected Findings
> 
> Impacts
> 
> Future Questions

### References

# **Introduction**

### What is Stellar Classification?

Our project leverages the Sloan Digital Sky Survey's seventeenth release (SDSS17), which includes a Stellar Classification Dataset with over 100,000 celestial observations, to distinguish between stars, galaxies, and quasars based on their spectral properties. Each type of celestial body has a distinct spectral signature: stars shine with a steady light, galaxies emit light from vast star collections, and quasars emit intense energy from supermassive black holes. By accurately classifying these entities, we gain a better understanding of the universe's structure and evolutionary processes.

### Classifying Celestial Objects & Our Question

**Question:** Can we predict whether a stellar object is a star, a galaxy, or a quasar based on their photometric ratios and redshifts?

### Dataset

Our data set comes from Kaggle and explores the classification of celestial objects into stars, galaxies, and quasars.
The variables for this dataset include observed characteristics of the object's electromagnetic radiation, its unique identifier, and identification labels of each observation.

### Determining Features through Literature Review

Research has proven **color indices** and **photometric redshifts** to be good metrics for celestial object classification (Wolf et al., 2017). Color indices (u-g, g-r, r-i, and i-z) are the differences between the magnitudes of the object in two different filters, representing the object's color. They help to reveal the physical properties of celestial objects (Elting et al., 2018). The redshift value is based on increases in radiation wavelength and points to an object's lifetime and motion.

# **Methodology**

1. Load Data
2. Clean & Wrangle Data
3. Exploratory Data Analysis
4. Visualize Exploratory Data
5. Data Analysis
> 5.1 Upsample uneven data
>
> 5.2 Standardize parameters
> 
> 5.3 Choosing the best k
> 
> 5.4 Fitting classifier with best k
>
> 5.5 Making predictions on testing data
6. Evaluating Results

#### **Preliminary exploratory data analysis** 

We first install and load the necessary packages for explanatory analysis.

In [3]:
# Run this cell if neccessary
install.packages('kknn')
install.packages("themis")
install.packages('cowplot')

Warning message in install.packages("kknn"):
“installation of package ‘kknn’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [4]:
# Run this cell to download all necessary packages before continuing
library(rvest)
library(tidyverse)
library(tidymodels)
library(tidyr)
library(dplyr)
library(ggplot2)
#library(kknn)
library(themis)
library(shiny)
library(cowplot)


Attaching package: ‘shiny’


The following object is masked from ‘package:infer’:

    observe



Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp




## **1) Load Data**

We read in our dataset from the URL generated through GitHub:

In [ ]:
URL <- 'https://raw.githubusercontent.com/jasukej/dsci-100-2024w2-group-19/main/data/star_classification.csv'
celestial_data <- read_csv(URL)
glimpse(celestial_data)

The total set of variables in this data set are:

1. **obj_ID**: Object Identifier, the unique value that identifies the object in the image catalog used by the CAS
2. **alpha**: Right Ascension angle (at J2000 epoch)
3. **delta**: Declination angle (at J2000 epoch)
4. **u**: Ultraviolet filter in the photometric system
5. **g**: Green filter in the photometric system
6. **r**: Red filter in the photometric system
7. **i**: Near Infrared filter in the photometric system
8. **z**: Infrared filter in the photometric system
9. **run_ID**: Run Number used to identify the specific scan
10. **rereun_ID**: Rerun Number to specify how the image was processed
11. **cam_col**: Camera column to identify the scanline within the run
12. **field_ID**: Field number to identify each field
13. **spec_obj_ID**: Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class)
14. **class**: object class (galaxy, star or quasar object)
15. **redshift**: redshift value based on the increase in wavelength
16. **plate**: plate ID, identifies each plate in SDSS
17. **MJD**: Modified Julian Date, used to indicate when a given piece of SDSS data was taken
18. **fiber_ID**: fiber ID that identifies the fiber that pointed the light at the focal plane in each observation

## **2) Clean & Wrangle Data**

The dataset is already tidy and contains no null values. 
* We remove all duplicate observations with the same `spec_obj_ID`, as they refer to the same celestial object. 
* Based on the literature review, we will mutate the data to **display color indices in order of the electromagnetic radiation spectrum**.
* To fit the data with our analysis, we establish the `class` as a categorical variable, and select only the relevant variables to our classification problem.

Our clean and wrangled data will be assigned to a new tibble called `tidy_celestial`.

In [ ]:
tidy_celestial <- celestial_data |>

# Keep only distinct observations with the same spectral object IDs
    distinct(spec_obj_ID, .keep_all = TRUE) |>

# Establish class as a categorical variable
    mutate(class = as_factor(class)) |>

# Create new columns for the ratio of color indices
    mutate(u_g = u - g,     # Ultraviolet - Green
           g_r = g - r,     # Green - Red
           r_i = r - i,     # Red - Near Infrared
           i_z = i - z)  |> # Near Infrared - Infrared 

# Filter missing values
    filter(!is.na(u_g + g_r + r_i + i_z + redshift)) |>

# Select only relevant features
    select(class, u_g, g_r, r_i, i_z, redshift) 

# Set table title
title <- "Table 1: Tidy Celestial Object Dataset"
cat("\n", title, "\n")
                    
head(tidy_celestial)

## **3) Exploratory Data Analysis**

#### **3.1) Split training and testing data**

First we split the `tidy_celestial` dataset into training `celestial_train` and `celestial_test` sets to ensure the classifier is unbiased towards data outside the training set (in this case, our testing data). We have chosen to use a 75/25 split; 75% of our data will be in the training set and 25% will be in the testing set. A larger proportion of training data has been found to be beneficial for the accuracy of the classifier.

We do not check for missing values as we have previously filtered them out.

In [ ]:
set.seed(2024)

celestial_split <- initial_split(tidy_celestial, prop = 0.75, strata = class)
celestial_train <- training(celestial_split)
celestial_test <- testing(celestial_split)

# checking datasets
glimpse(celestial_train)
glimpse(celestial_test)

#### **3.2) Proportions of categorical data**

We verify that the `class` column in our data contains a roughly equal number of positive and negative responses in order to guarantee a reliable and unbiased classifier. To accomplish this, we count each category in the `class` column and use a bar graph to visualize the proportions.

In [ ]:
# Finding distribution of classes
options(repr.plot.height = 5, repr.plot.width = 5)
training_count <- celestial_train |> group_by(class) |> summarize(count = n())

training_count_plot <- training_count |> 
                        ggplot(aes(x = class, y = count, fill = class)) + 
                        geom_bar(stat = "identity") + 
                        geom_text(aes(label = count), vjust = -0.5, position = position_dodge(width = 0.9)) +
                        labs(x = "Object type", y = "Count") +
                        ggtitle("Figure 1: Celestial Objects Detected") + 
                        theme(text = element_text(size = 14))
training_count_plot

We will remove one abnormality in our distribution.

In [ ]:
# Identify abnormalities in distribution
celestial_train |> filter(i_z > 10000)

In [ ]:
celestial_train <- celestial_train |> filter(i_z < 10000)

#### **3.3) Parameter distributions**

Next, we compute the summary statistics of the parameters `u_g`, `g_r`, `r_i`, `i_z`, and `redshift` to check if we will need to standardize our data.

In [ ]:
# Table for Maximum Values
max_table <- celestial_train |>
    group_by(class) |>
    summarize(across(c(u_g, g_r, r_i, i_z, redshift), 
                   ~ max(., na.rm = TRUE), 
                   .names = "max_{.col}"))

# Table for Minimum Values
min_table <- celestial_train |>
    group_by(class) |>
    summarize(across(c(u_g, g_r, r_i, i_z, redshift), 
                   ~ min(., na.rm = TRUE), 
                   .names = "min_{.col}"))

# Table for Median Values
mean_table <- celestial_train |>
    group_by(class) |>
    summarize(across(c(u_g, g_r, r_i, i_z, redshift), 
                   ~ mean(., na.rm = TRUE), 
                   .names = "mean_{.col}"))

max_table
min_table
mean_table

### **Summary of Exploratory Data Analysis**

* From the bar plot displaying counts of celestial objects, we see that our training data is **uneven** and must be **upsampled** when training our algorithm.

* The summary statistics show distinct differences between the mean predictor values between classes. The max and min values verify that there are no longer any problematic outliers in the data. 

## **4) Visualizing relationships**

We will now visualize our results using a **box plot** displaying the correlation between the predictors and the response variable `class`.

In [ ]:
# Pivot to display predictors beside the class
predictor_distribution <- pivot_longer(celestial_train, -class, names_to = "predictor", values_to = "value")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)

# Plotting each distribution
training_boxplots <- ggplot(predictor_distribution, aes(x = class, y = value, fill = class)) + 
                        geom_boxplot() +  
                        facet_wrap(~ predictor, scales = "free_y") +  # Each predictor is its own plot
                        labs(y = "Value", x = "Class") + 
                        ggtitle("Distribution of predictors per class") +
                        theme_minimal() +
                        theme(plot.title = element_text(size = 18, face = "bold", hjust = 0.5),
                              axis.title = element_text(size = 18),
                              axis.text = element_text(size = 12), 
                              axis.text.x = element_text(angle = 45, size = 12),
                              strip.text = element_text(size = 15),
                              legend.title = element_text(size = 14, face = "bold"),
                              legend.text = element_text(size = 12))
training_boxplots

Looking at the box plots, despite some overlap in the color indices, the distribution of predictor values for each class is **distinct** when compounded with the redshift values. 

## **5) Data Analysis**

As we have already split the training and testing datasets, we will first upsample the uneven data, then standardize our parameters, and create our K-nearest neighbour classifier with celestial_train. The following is a workflow of our methods:

**5.1) Upsample uneven data**

> a) Pre-upsampling visualization
>
> b) Upsample the data
>
> c) Post-upsampling visualization

**5.2) Standardize parameters**

**5.3) Choosing a k**

> a) Create 5 splits
>
> b) Create and tune k-classification spec
> 
> c) Compute mean accuracies of k = [15,40] to avoid overfitting and underfitting

**5.4) Fitting classifier with determined best k**

**5.5) Testing classifier on testing data**

#### **5.1) Upsampling uneven data**
> a) Pre-upsampling visualization
>
> b) Upsample the data
>
> c) Post-upsampling visualization

#### **a) Pre-upsampling visualization**

We visualize the data before upsampling; the plot for the raw training data will be named `not_ups_plot`.

In [ ]:
options(repr.plot.height = 5, repr.plot.width = 10)

#Assign training data
not_ups_celestial <- celestial_train

# 

#### **b) Upsampling data**

We upsample the data in this step, and display the count of each class in `ups_count`.

In [ ]:
set.seed(2024)

# themis
# Creating recipe for upsampling
ups_recipe <- recipe(class ~ u_g + g_r + r_i + i_z + redshift, data = celestial_train) |>
                step_smote(class, over_ratio = 1, skip = FALSE) |>
                prep()

# Applying this to our training data
celestial_train_ups <- bake(ups_recipe, celestial_train)

# Check that our data is upsampled
ups_count <- celestial_train_ups |> 
                group_by(class) |>
                summarize(count = n())

# Set table title
title <- "Table 4: Proportions of Celestial Class after Upsampling"
cat("\n", title, "\n")

# Print table
ups_count

#### **c) Post-upsampling visualization**

We visualize the data after upsampling. The plot for the upsampled training data will be named `upsampled_plot`.

Looking at the data before and after upsampling...

From our visualization, we see that upsampling our data balances it and classifications are more sensitive to changes. This improves the recall of our model.

### **5.2) Standardize Parameters**

From our exploratory data analysis, we found that the standard deviations of the predictors differed by a considerable amount. As the KNN classification algorithm is sensitive to differences in scale between predictors, we will standardize the predictors through centering and then scaling the data. 

We will only apply standardization to the training set for now to ensure that the testing data does not influence our model whatsover. We will apply standardization separately before predicting the results of our testing data.

In [ ]:
# Mean and standard deviation before standardization

# Standardizing the data 
celestial_recipe <- recipe(class ~ u_g + g_r + r_i + i_z + redshift, data = celestial_train) |>
                    step_scale(all_predictors()) |>
                    step_center(all_predictors()) |>
                    prep()

scaled_train <- bake(celestial_recipe, celestial_ups_train)

# Mean and standard deviation after standardization


### **5.3) Choosing a k**

To maximize the accuracy of our classifier, we will create 5 different splits of our training data and predict the mean accuracy across the splits for k = [1, 50]. We will then plot a line plot displaying the accuracy using the respective k neighbours.

#### **a) Creating 5 splits**

#### **b) Creating specification for knn classfication**

#### **c) Computing mean accuracies of k = [15,40]**

### **5.4) Fit classifier with best k found**

### **5.5) Make predictions on testing data**

## **6) Evaluating results**

#### **a) Visualizing the Confusion Matrix**

## **Discussion**

### Summary of Results

### Expected Findings

### Impacts

### Future Implications

# **References** 

fedesoriano. (2022). Stellar Classification Dataset - SDSS17. Kaggle.com. https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17/data

Wolf, C., Meisenheimer, K., Kleinheinrich, M., Borch, A., Dye, S., Gray, M., Wisotzki, L., Bell, E., Rix, H., Cimatti, A., Hasinger, G., & Szokoly, G. (2004). A catalogue of the Chandra Deep Field South with multi-colour classification and photometric redshifts from COMBO-17. Astronomy and Astrophysics, 421, 913-936. https://doi.org/10.1051/0004-6361:20040525.

Elting, C., C. Bailer-Jones, & Smith, K. (2018). Photometric Classification of Stars, Galaxies and Quasars in the Sloan Digital Sky Survey DR6 Using Support Vector Machines. https://www.semanticscholar.org/paper/Photometric-Classification-of-Stars%2C-Galaxies-and-Elting-Bailer-Jones/7e880193b3a75c789c7352dd65743666af8e4dab